# A multi-agent coding project (iteration 2)

This multi-agent coding project attempts to demonstrate how a number of AI Agents can be employed as a team to get basic coding done quicker.

## The scenario: working together to write the word "cat"

Build a team of multiple agents:
1. **CatCoordinatorBot:** stores the overall state of the project, and creates instructions for the other workers. Knows the overall plan and how to work through it.
2. **CatLetterBot:** returns letters in "cat" from a an index integer.
3. **CatCheckerBot:** checks that a number is the correct index for a letter in "cat", returns "Yes" or "No".

## Interactions

All interactions between Bots are mediated inside the `MultiAgentProject` class. This receives a dictionary of agents made up of:
- `default`: the default agent to use
- `ids`: a dictionary of `name: openai_assistant_id` key-value pairs

The default agent is asked to "Start" when you call the `.run()` method on an instance of `MultiAgentProject`. The entire project should then run without interaction from the user.

## Response payloads

There is a basic JSON payload used for responses from each agent, as follows:

```
{
    "msg_status": "(success|failure)",
    "msg_content": <content>,
    "msg_report": <log_messages>,
    "msg_target": null|<agent>
}
```

## Choice of agents: OpenAI Assistants (based on `gpt-4o-mini`)

We'll prompt the agents with as little information as required to get the job done. We will need to create the Agents, in this case as OpenAI _Assistants_, using the base LLM `gpt-4o-mini`.

### Setting the temperature (randomness) low!

The OpenAI Assistants have a setting `temperature`, which controls randomness. The default is 1. As the temperature approaches 0, the model becomes more deterministic and repetitive. We will set `temperature=0.75`.

### Set-up

Import relevant libraries and set up `OPENAI_API_KEY` environment variable which authenticates to the OpenAI APIs.

In [2]:
import os, time, json
from IPython import display
from openai import OpenAI

keyfile = os.path.expanduser(f"~/.openai-skills-evalution-api-key")
os.environ["OPENAI_API_KEY"] = open(keyfile).read().strip()

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

client = OpenAI()

### Create the Agents

In [44]:
# We'll store the IDs in here to avoid creating lots of duplicate agents
agents = {
    "default": "CatCoordinatorBot",
    "ids": {
'CatCheckerBot': 'asst_SeK2QTmawhAMeA472isCJl2O',
 'CatLetterBot': 'asst_vw03ZwMXRQRwctWOrU6E31BH',
 'CatCoordinatorBot': 'asst_Ki6VvrfwokpmsBROCp5Qjf8A'
    }
}

agent_prompts = {
    "CatCoordinatorBot": """
You are an agent who has a specific responsibility to build a string of 3 characters that represent the word "cat".
You can ONLY DO THIS BY CREATING INSTRUCTIONS FOR OTHER AGENTS. 

All your responses should be in JSON FORMAT, as follows:
{"msg_status": "success" or "failure",
 "msg_target": null or <agent to send to>,
 "msg_content": <content>,
 "msg_report": <report>}

Your response will be parsed by code in the client, which will then send that message to the Agent specified.
IMPORTANT: Each time you send an instruction to another agent, the next message you receive will be forwarded from that Agent that 
you gave the instruction to.

Whenever you get an instruction to REPORT, then include the report text in the "msg_report" section of the response, so that the 
user can get an update on progress. BUT DO NOT SEND THE RESPONSE UNTIL YOU HAVE DONE THE ACTUAL TASK.

The 2 OpenAI assistants that you can interact with are:
 - CatLetterBot
 - CatCheckerBot

Start work when you receive a message from the user saying: "Start".
IMPORTANT!!! When you receive the message "Start", please reset any state memory that you have related to your main task.
 
You will start by storing an empty string. Then you will repeat the following task 3 times, once for each character in the word "cat":
  1. Take a number $N. It will be 1 for the first iteration, 2 for the second iteration, all the way up to 3 for the final iteration.
  2. REPORT: "Sending number $N to CatLetterBot"
  3. Write an instruction for the assistant called `CatLetterBot`, containing only the number $N in the contents. 
     DO NOT SEND IT ANY OTHER TEXT OR CONTENT!
  4. `CatLetterBot` will return a single letter, $L, in its response.
  5. REPORT: "CatLetterBot returned: $L"
  6. If `CatLetterBot` responds with anything not in "cat", then return: "SORRY, WE FAILED!", followed by the number $N. 
     STOP WORK IF THIS HAPPENS!
  7. REPORT: "Sending $N, $L to CatCheckerBot"
  8. Write an instruction for the assistant called `CatCheckerBot`, containing only $N and $L that was returned by `CatLetterBot`
     in the contents. The two characters should be separated be a space. FOLLOW THIS INSTRUCTION EXACTLY.
  9. REPORT: "CatCheckerBot returned: ${response from CatLetterBot}"
  10. If `CatCheckerBot` responds with "No", then stop the process and respond with "SORRY, WE FAILED!", followed by the number $N 
      and the single letter returned by `LetterBot`. STOP WORK IF THIS HAPPENS!
  11. `CatCheckerBot` must only return "No" or "Yes", so we assume it has returned "Yes" if we get here. 
  12. REPORT: "Added $L to my result"
  13. Append the letter returned by `CatLetterBot` to your current string of characters.
  14. If less than 3 characters have been added to the string, start the next iteration of the loop.
  15. If all 3 characters have been added to the string, REPORT: "My work is done! Result: ${the complete string}, with status as "success".
""",

    "CatLetterBot": """
You are an agent who ONLY KNOWS ABOUT THE LETTERS IN THE WORD "cat" AND THE CORRESPONDING 3 NUMBERS THAT INDEX THEM.
Your job is to translate a number (index) into a lower-case letter from the word "cat".

All your responses should be in JSON FORMAT, as follows:
{"msg_status": "success" or "failure",
 "msg_content": <content>}

When you receive a message, do the following:
1. If it is not a number between 1 and 3: return as a failure with content: "INVALID INPUT: <input>".
2. If it is a number between 1 and 3: RETURN THE LETTER CORRESPONDING TO THAT NUMBER WHEN INDEXED FROM 1 as the content.

NOTE: If you need help with counting in order to work out the index, you can use this Python code:
`letter = "cat"[number - 1]` 
""",

    "CatCheckerBot":  """
You are an agent who ONLY KNOWS ABOUT THE LETTERS IN THE WORD "cat" AND THE CORRESPONDING 3 NUMBERS THAT INDEX THEM.
Your job is to check that a number (index) corresponds to a letter in the word "cat".

All your responses should be in JSON FORMAT, as follows:
{"msg_status": "success" or "failure",
 "msg_content": <content>}

IT IS IMPORTANT TO KNOW THAT THE INDEXING STARTS AT 1 (NOT 0)!

When you receive a message, do the following:
1. Remove any punctuation from the input text you recieve.
2. If you have not received a number (between 1 and 3 inclusive) and a letter (from "cat"), then return a failure with content: "INVALID INPUT: <input>".
3. Otherwise, check that the number is the correct index value to find the letter in the word "cat" (when indexing from 1).
4. If correct: RETURN a success with content: "Yes".
5. If incorrect: RETURN a success with content: "No".

NOTE: If you need help calculating whether the index and letter correspond, you can use the Python code:
`result = "Yes" if "cat"[number - 1] == letter else "No"`
"""
}

In [37]:
def delete_agents(agent_prompts=agent_prompts):
    """
    Delete agents before remaking.
    """
    # Get current IDs
    existing_assistant_ids = get_assistant_ids()
    
    for agent_name, prompt in agent_prompts.items():
        assistant_id = agents["ids"].get(agent_name, "")
    
        if assistant_id and assistant_id in existing_assistant_ids:
            assistant = client.beta.assistants.delete(assistant_id)
            agents["ids"][agent_name] = ""
            print(f"Deleted agent: '{agent_name}'")
        else:
            print(f"Agent '{agent_name}' does not exist (so no need to delete).")

In [38]:
DO_DELETE_AGENTS = False
if DO_DELETE_AGENTS:
    delete_agents()

#### NOTE: Assistants are persitent so test for their IDs and update the agent_data dict accordingly


This stops us creating lots of identical OpenAI Assistants!

In [39]:
def get_assistant_ids(with_names=False, match=agent_prompts):
    my_assistants = client.beta.assistants.list(
        order="desc",
        limit="50",
    )

    if with_names:
        return {asst.name: asst.id for asst in my_assistants.data if match and asst.name in match}
    else:
        return [asst.id for asst in my_assistants.data if match and asst.name in match]

In [40]:
get_assistant_ids(True)

{}

In [41]:
def create_agents(agent_prompts=agent_prompts):
    # Get current IDs
    existing_assistant_ids = get_assistant_ids(with_names=True, match=agent_prompts)
    
    for agent_name, prompt in agent_prompts.items():
        assistant_id = agents["ids"].get(agent_name, "")
    
        if not assistant_id or assistant_id not in existing_assistant_ids:
            print(f"Creating agent: {agent_name}")
            assistant = client.beta.assistants.create(
                name=agent_name,
                instructions=prompt.strip(),
                model="gpt-4o-mini",
                temperature=0.75,
                response_format={"type": "json_object"})

            agents["ids"][agent_name] = assistant.id
        else:
            print(f"Agent '{agent_name}' already exists")

In [42]:
create_agents()

Creating agent: CatCoordinatorBot
Creating agent: CatLetterBot
Creating agent: CatCheckerBot


In [43]:
get_assistant_ids(True)

{'CatCheckerBot': 'asst_SeK2QTmawhAMeA472isCJl2O',
 'CatLetterBot': 'asst_vw03ZwMXRQRwctWOrU6E31BH',
 'CatCoordinatorBot': 'asst_Ki6VvrfwokpmsBROCp5Qjf8A'}

### Now we have a set of agents ready to do their work

## Getting agents to talk to each other

From the investigations I have done so far, you cannot just tell an OpenAI Assistant to *talk to* another OpenAI Assistant. You need to **write some code to glue them together**. 

In [46]:
class MultiAgentProject:
    
    def __init__(self, agents, retries=0, DEBUG=False, completion_message="WORK COMPLETED"):
        self.agents = agents
        self.DEBUG = DEBUG
        self.history = []
        self.retries = retries
        self.completion_msg = completion_message
        self.reread_msg = "Please re-read your instructions and try again.\n"

    def run(self):
        self.thread = client.beta.threads.create()
        print(f"Working on thread: {self.thread.id}")

        resp = self.dispatch({"msg_target": self.agents["default"], "msg_content": "Start"})

        while self.completion_msg not in resp.get("msg_report", ""):
            resp = self.dispatch(resp)
            if resp.get("msg_status", "") != "success":
                break

        print(f"Final message:  {resp}")

    def dispatch(self, resp):
        agent = resp.get("msg_target", self.agents["default"])
        content = resp.get("msg_content", "")
        prefix = ""

        if self.DEBUG: 
            print(f"Preparing to send to {agent}: {content}")

        # Do as many retries as allowed (if not successful OR no content returned)
        for retry in range(self.retries + 1):
            print(f"Retry: {retry}:: ", end="")
            if retry > 0:
                prefix = self.reread_msg
                
            resp = self.send_message(agent, f"{prefix}{content}")
            if resp.get("msg_status", "") == "success" and (
                    resp.get("msg_content", "") or 
                    self.completion_msg in resp.get("msg_report", "")):
                break
                
        return resp

    # Waiting in a loop
    def wait_on_run(self, run):
        while run.status == "queued" or run.status == "in_progress":
            run = client.beta.threads.runs.retrieve(thread_id=self.thread.id, run_id=run.id)
            if self.DEBUG: print(f"Polling for run: {run.id}")
            time.sleep(1)
        return run
            
    def log(self, agent, message):
        print(f"[INFO] Response from {agent}: {message}")

    def get_response(self):
        return client.beta.threads.messages.list(thread_id=self.thread.id, order="desc")
    
    def send_message(self, agent, message):
        client.beta.threads.messages.create(
            thread_id=self.thread.id, role="user", content=message
        )

        print(f"Sending to '{agent}': {message}")
        run = client.beta.threads.runs.create(
            thread_id=self.thread.id, assistant_id=self.agents["ids"][agent]
        )
        run = self.wait_on_run(run)

        response = self.get_response()
        if self.DEBUG: 
            print("RESP:", [i.text.value for j in response.data for i in j.content])

        message_content = json.loads(response.data[0].content[0].text.value)
        
        self.log(agent, message_content)
        return message_content


p = MultiAgentProject(agents, retries=2, DEBUG=False, completion_message="My work is done")
p.run()

Working on thread: thread_13ckeQnNHV25IeFylXqnfZsc
Retry: 0:: Sending to 'CatCoordinatorBot': Start
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_target': 'CatLetterBot', 'msg_content': '1', 'msg_report': 'Sending number 1 to CatLetterBot'}
Retry: 0:: Sending to 'CatLetterBot': 1
[INFO] Response from CatLetterBot: {'msg_status': 'success', 'msg_content': 'c'}
Retry: 0:: Sending to 'CatCoordinatorBot': c
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_target': 'CatCheckerBot', 'msg_content': '1 c', 'msg_report': 'CatLetterBot returned: c'}
Retry: 0:: Sending to 'CatCheckerBot': 1 c
[INFO] Response from CatCheckerBot: {'msg_status': 'success', 'msg_content': 'Yes'}
Retry: 0:: Sending to 'CatCoordinatorBot': Yes
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_report': 'Added c to my result'}
Retry: 1:: Sending to 'CatCoordinatorBot': Please re-read your instructions and try again.
Yes
[INFO] Response from CatCoordinato

### Example output - including retries

```
Working on thread: thread_QDYgkGUOiu0pXMtRluLwk1X2
Retry: 0:: Sending to 'CatCoordinatorBot': Start
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_target': 'CatLetterBot', 'msg_content': '1', 'msg_report': 'Sending number 1 to CatLetterBot'}
Retry: 0:: Sending to 'CatLetterBot': 1
[INFO] Response from CatLetterBot: {'msg_status': 'success', 'msg_content': 'c'}
Retry: 0:: Sending to 'CatCoordinatorBot': c
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_target': 'CatCheckerBot', 'msg_content': '1 c', 'msg_report': 'CatLetterBot returned: c'}
Retry: 0:: Sending to 'CatCheckerBot': 1 c
[INFO] Response from CatCheckerBot: {'msg_status': 'success', 'msg_content': 'Yes'}
Retry: 0:: Sending to 'CatCoordinatorBot': Yes
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_report': 'Added c to my result'}
Retry: 1:: Sending to 'CatCoordinatorBot': Please re-read your instructions and try again.
Yes
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_target': 'CatLetterBot', 'msg_content': '2', 'msg_report': 'Sending number 2 to CatLetterBot'}
Retry: 0:: Sending to 'CatLetterBot': 2
[INFO] Response from CatLetterBot: {'msg_status': 'success', 'msg_content': 'a'}
Retry: 0:: Sending to 'CatCoordinatorBot': a
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_target': 'CatCheckerBot', 'msg_content': '2 a', 'msg_report': 'CatLetterBot returned: a'}
Retry: 0:: Sending to 'CatCheckerBot': 2 a
[INFO] Response from CatCheckerBot: {'msg_status': 'failure', 'msg_content': 'INVALID INPUT: Yes'}
Retry: 1:: Sending to 'CatCheckerBot': Please re-read your instructions and try again.
2 a
[INFO] Response from CatCheckerBot: {'msg_status': 'success', 'msg_target': 'CatCheckerBot', 'msg_content': '3 t', 'msg_report': 'CatLetterBot returned: t'}
Retry: 0:: Sending to 'CatCheckerBot': 3 t
[INFO] Response from CatCheckerBot: {'msg_status': 'success', 'msg_content': 'Yes'}
Retry: 0:: Sending to 'CatCoordinatorBot': Yes
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_report': 'Added t to my result'}
Retry: 1:: Sending to 'CatCoordinatorBot': Please re-read your instructions and try again.
Yes
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_report': 'My work is done! Result: cat'}
Retry: 2:: Sending to 'CatCoordinatorBot': Please re-read your instructions and try again.
Yes
[INFO] Response from CatCoordinatorBot: {'msg_status': 'success', 'msg_report': 'My work is done! Result: cat'}
Final status:  success
Final message: None
```

### Afterthoughts

1. The general framework/pattern of the code could be tidied and re-used in any Multi-Bot interaction.
2. This approach would also work with different third-party providers. I.e. if you moved from OpenAI to Gemini or Meta, you could keep the same approach.
3. The individual assistants have limitations, that might be easiest to work around, e.g.:
    - Cannot push/pull to/from GitHub


You could generalise things as follows:
1. The main class could be `MultiAgentProject`.
2. A single agent could be the `Coordinator`.
3. All other agents can respond with either:
   - a simple text response, e.g.: `A polar bear`
   - or an instruction to forward to response, e.g.: `Send to ArcticExplorer: A polar bear`
   - NOTE: This approach will allow for more nuanced interactions where some agents can direct their outputs straight to another.
4. Needs better failure handling and retry capabilities:
   - Maybe all agents can accept something like: `"Re-read your initial prompting, and try again."`
5. Needs a history object so that all history is accessible.
6. All complex stuff that just needs generic code can sit in the client package, e.g.:
   - Interactions with GitHub
   - Simple actions that Python can do
7. Input files can be uploaded to the workspace associated with the Assistants, so this might be appropriate.